In [ ]:
options(dplyr.summarise.inform = FALSE)
library(tidyverse)
library(data.table)
library(survival)
library(gridExtra)

In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(dirname(dirname(dirname(getwd()))),'/stats.r'))

#### 0 - Run analyses

In [ ]:
system.time(go <- readRDS(paste0(SHARE_DIR, "ready_ex.Rds")))

In [ ]:
df <- go$data_ready %>% filter(clin_anti_pd_treated == 1)
features <- go$features

#### 1 - Go for a run

- Non-Responders

In [ ]:
results <- data.frame()
system.time(
for( i in features){
    results <- rbind(results, 
                     get_stats2( y = "Y_nr", 
                                 x = i, 
                                 covariate = " + as.factor(location) + clin_age + clin_sex", 
                                 data = "df", 
                                 model = "bor"))
})

In [ ]:
response <- data.frame()
system.time(
for( i in features){
    response <- rbind(response, 
                      get_stats2( y = "Y_bor", 
                                 x = i, 
                                 covariate = " + as.factor(location) + clin_age + clin_sex", 
                                 data = "df", 
                                 model = "bor"))
})

In [ ]:
ordinal_response <- data.frame()
system.time(
for( i in features){
    ordinal_response <- rbind(ordinal_response, 
                      get_stats2( y = "Y_ro", 
                                 x = i, 
                                 covariate = " + as.factor(location) + clin_age + clin_sex", 
                                 data = "df", 
                                 model = "lm"))
})

In [ ]:
survive <- data.frame()
system.time(
for( i in features){
    survive <- rbind(results, 
                     get_stats2( y = "Surv(Y_os_days, Y_os_event)", 
                                 x = i, 
                                 covariate = " + as.factor(location) + clin_age + clin_sex + purity", 
                                 data = "df", 
                                 model = "coxph"))
})

#### 2 - Figure examples

In [ ]:
plotter <- function( results, x_lab = "Log Odds Estimate", title = "Non-Response vs features (adjusted)"){
  ggplot( results, aes( x = est, y = -log10(pval), color = type)) + 
      geom_point() + 
      theme_classic() + 
      xlab(x_lab) + 
      ylab("-Log10 (p-value)") + 
      ggtitle(title) + 
      geom_hline(yintercept = -log10(.01)) + 
      xlim(-3,3) + 
      ylim(0,10) +
      facet_wrap(~type, ncol = 4) + 
      theme(legend.position = "none", plot.title = element_text(hjust = .5))
}

In [ ]:
labs <- c("purity_tmbStatus" = "TMB", "rna_geneset_gene_set_t_cell_gep_6" = "T-cell", "clin_hasSystemicPreTreatment" = "Pretreated")

In [ ]:
a <- 
plotter(results) + 
  geom_text( data = results %>% filter(x %in% names(labs)) %>% rowwise() %>% mutate( lab = labs[[x]]) %>% ungroup(), 
             aes(x = est, y = -log10(pval) + 1.5, label = lab))

In [ ]:
b <- 
plotter(response, title = "Best Overall Response vs features (adjusted)") + 
  geom_text( data = response %>% filter(x %in% names(labs)) %>% rowwise() %>% mutate( lab = labs[[x]]) %>% ungroup(), 
             aes(x = est, y = -log10(pval) + 1, label = lab))

In [ ]:
c <- 
plotter(survive, title = "Overall Survival vs features (adjusted)") + 
  geom_text( data = survive %>% filter(x %in% names(labs)) %>% rowwise() %>% mutate( lab = labs[[x]]) %>% ungroup(), 
             aes(x = est, y = -log10(pval) + 1, label = lab))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
grid.arrange( a, b,c, ncol = 2)

#### 3 - Other study

In [ ]:
#df %>% select(contains("pret"))

In [ ]:
little <- 
df %>% 
  filter(location == "Lung") %>% 
  transmute(Y_bor = ifelse(Y_bor == 1, TRUE, FALSE), 
            Y_nr, 
            pretreated = ifelse(clin_hasSystemicPreTreatment < 0, FALSE, TRUE), 
            tmb_low = ifelse(purity_tmbStatus < 0, TRUE, FALSE), 
            kras_driver = ifelse(driver_KRAS > 0,TRUE, FALSE),
            keap1 = ifelse(driver_KEAP1 > 0,1, 0),
            egfr = ifelse(driver_EGFR > 0,1, 0),
            stk11 = ifelse(driver_STK11 > 0,1, 0), 
            keap1_egfr_stk11_driver = ifelse(keap1 + egfr + stk11 > 0, TRUE, FALSE))

In [ ]:
summary(glm(Y_bor ~ tmb_low + keap1_egfr_stk11_driver, data = little))

In [ ]:
table(little %>% select(keap1_egfr_stk11_driver, Y_nr, tmb_low))

In [ ]:
table(little %>% select(keap1_egfr_stk11_driver, Y_bor, tmb_low))

In [ ]:
table(little %>% select(keap1_egfr_stk11_driver, Y_bor, pretreated, tmb_low))

In [ ]:
table(little %>% select(keap1_egfr_stk11_driver, pretreated))

#### 4 - See association to KEAP1, EGFR, STK11 drivers

In [ ]:
table(little %>% select(kras_driver, pretreated))